# Entity Resolution project @ Wavestone
## Create datasets

> *Datasets information from [here](Datasets.md) \
> Description to do but only take the raw data because the not raw data was already pre-processed*

> **Tristan PERROT**


## Import libraries

In [13]:
import os

while 'model' not in os.listdir():
    os.chdir('..')

In [14]:
import pandas as pd
import pickle

from model.utils import get_entity, serialize_entities

## Data

In [15]:
SET_1 = 'walmart'
SET_2 = 'amazon'

In [20]:
COLS_A_TO_REMOVE = ['id', 'upc']
COLS_B_TO_REMOVE = ['url', 'asin']

### Load data

In [16]:
DATA_NAME = f'{SET_1}-{SET_2}'

In [17]:
DATA_DIR = os.path.join('data', DATA_NAME)

In [18]:
table_a = pd.read_csv(os.path.join(DATA_DIR, f'1_{SET_1}.csv'), encoding='latin1')
table_b = pd.read_csv(os.path.join(DATA_DIR, f'2_{SET_2}.csv'), encoding='latin1')
pairs_train = pd.read_csv(os.path.join(DATA_DIR, 'gs_train.csv'), encoding='latin1')
pairs_val = pd.read_csv(os.path.join(DATA_DIR, 'gs_val.csv'), encoding='latin1')
pairs_test = pd.read_csv(os.path.join(DATA_DIR, 'gs_test.csv'), encoding='latin1')

### Pre-processing

In [19]:
display(table_a.head())
display(table_b.head())

# Display matches
display(pairs_train.head())

,subject_id,id,upc,brand,groupname,title,price,shelfdescr,shortdescr,longdescr,imageurl,orig_shelfdescr,orig_shortdescr,orig_longdescr,modelno,shipweight,dimensions
0,1,14249992,64109216245,Draper,Electronics - General,Draper Infrared Remote Transmitter,58.45,Infrared transmitter. 3-button operation for i...,NaN,DR1143Infrared transmitter. 3-button operation...,http://i.walmartimages.com/i/mp/00/64/10/92/16...,Infrared transmitter. 3-button operation for i...,NaN,DR1143Infrared transmitter. 3-button operation...,121066,2.00,NaN
1,2,10928662,1034383417,Epson,Monitors,Epson 1500 Hours 200W UHE Projector Lamp ELPLP12,438.84,NaN,Epson ELPLP12 Replacement Lamp,EPSON ELPLP12 1500HRS 200V REPL LAMP FOR LAMP ...,http://i.walmartimages.com/i/p/00/01/03/43/83/...,NaN,Epson ELPLP12 Replacement Lamp,EPSON ELPLP12 1500HRS 200V REPL LAMP FOR LAMP ...,ELPLP12,0.95,6.75 x 5.75 x 5.5 inches
2,3,11961447,80844700031,Comprehensive,TV Accessories,Comprehensive Two-Piece 75 Precision BNC Jack ...,59.25,Comprehensive s True 75 connectors eliminate i...,NaN,CH1151Comprehensive s True 75 connectors elimi...,http://i.walmartimages.com/i/mp/00/80/84/47/00...,Comprehensive's True 75 connectors eliminate i...,NaN,CH1151Comprehensive's True 75 connectors elimi...,BJ-2C7559,0.05,NaN
3,4,13044637,79006932160,D-Link,Garden - General,D-Link DCS-1100 Network Camera,99.82,Surveillance Network Camera Built-in Omni-dir...,The D-Link DCS-1100 Network Camera comes with ...,Surveillance Network Camera Built-in Omni-dir...,http://i.walmartimages.com/i/p/00/79/00/69/32/...,<UL><LI>Surveillance/Network Camera</LI><LI>Bu...,The D-Link DCS-1100 Network Camera comes with ...,<UL><LI>Surveillance/Network Camera</LI><LI>Bu...,DCS-1100,NaN,NaN
4,5,13214131,6503081691,StarTech,Electronics - General,StarTech.com RKPW247015 24 Outlet Power Strip,59.00,Fits most 42U racks and equipment cabinets 10...,The StarTech.com RKPW247015 24 Outlet Power St...,24 Outlet Power Strip solution for your rackmo...,http://i.walmartimages.com/i/p/00/06/50/30/81/...,<ul><li>Fits most 42U racks and equipment cabi...,The StarTech.com RKPW247015 24 Outlet Power St...,<ul><li>24 Outlet Power Strip solution for you...,RKPW247015,5.25,72.5 x 2.5 x 1.5 inches


,subject_id,url,asin,brand,modelno,category1,pcategory1,category2,pcategory2,title,listprice,price,prodfeatures,techdetails,shortdescr,longdescr,dimensions,imageurl,itemweight,shipweight,orig_prodfeatures,orig_techdetails,groupname
0,1,http://www.amazon.com/Koss-EQ50-3-Band-Stereo-...,B00005UPF6,Koss,152132,Headphone Accessories,Accessories Supplies,Equalizers,Home Audio Theater,Koss EQ50 3-Band Stereo Equalizer,19.99,12.65,NaN,10dB boost or cut per channel Unity gain ampli...,The Koss EQ50 is a three band stereo equalizer...,The pocket-size Koss 3-Band Equalizer delivers...,7.6 x 4.8 x 1.4 inches,http://ecx.images-amazon.com/images/I/4124dZb2...,3.2 ounces,1 pounds,NaN,<ul><li>10dB boost or cut per channel</li> <li...,Headphone Accessories Accessories Supplies Equ...
1,2,http://www.amazon.com/Kodak-Black-Ink-Cartridg...,B0035JJJ96,Kodak,1163641,Inkjet Printer Ink,Printer Ink Toner,NaN,NaN,Kodak Black Ink Cartridge 10B 1163641,NaN,10.28,NaN,Ships in foil packaging,NaN,Kodak Black Ink Cartridge 10B is a standard bl...,5.3 x 3.2 x 1.2 inches,http://ecx.images-amazon.com/images/I/41%2B2Ea...,1.9 ounces,1 pounds,NaN,<ul><li>Ships in foil packaging</li> </ul>,Inkjet Printer Ink Printer Ink Toner
2,3,http://www.amazon.com/Kingston-128MX64-PC2700-...,B00008XOKZ,Kingston,KTC-D320/1G,Computers Accessories,Electronics,NaN,NaN,Kingston 128MX64 PC2700 COMPAQ Evo D320 KTC-D3...,60.00,33.75,NaN,DC341A,NaN,1GB - 333MHz DDR333 PC2700 - DDR SDRAM - 184-p...,6.8 x 2.2 x 0.5 inches,http://ecx.images-amazon.com/images/I/21D2P6AN...,1.3 ounces,1 pounds,NaN,<ul><li># DC341A</li> </ul>,Computers Accessories Electronics
3,4,http://www.amazon.com/Kinamax-MS-UES2-Precisio...,B000HJ75MK,Kinamax,MS-UES2,Mice,Keyboards Mice Input Devices,NaN,NaN,Kinamax MS-UES2 Mini High Precision USB 3-Butt...,NaN,6.99,NaN,Support Inquiries support sabrent.com Features...,NaN,KINAMAX MS-UES2 Mini High Precision USB 3-Butt...,4.3 x 2.8 x 1.3 inches,http://ecx.images-amazon.com/images/I/31gI-p6d...,1.6 ounces,1 pounds,NaN,<ul><li>Support Inquiries: support@sabrent.com...,Mice Keyboards Mice Input Devices
4,5,http://www.amazon.com/Kensington-K72349US-Wire...,B002LARRY4,Kensington,K72349US,Mice,Keyboards Mice Input Devices,NaN,NaN,Kensington K72349US Wireless Mouse for Netbooks,24.99,24.00,NaN,Stay plugged in with a nano receiver Web surfi...,NaN,Wireless MOUSE FOR NETBOOKS USBWRLS,4.1 x 3.2 x 1.5 inches,http://ecx.images-amazon.com/images/I/31eIBC9A...,2.1 ounces,1 pounds,NaN,<ul><li>Stay plugged in with a nano receiver</...,Mice Keyboards Mice Input Devices


,source_id,target_id,matching
0,624,2862,False
1,1510,16351,False
2,102,9859,False
3,839,13024,False
4,971,5512,False


In [21]:
table_a.rename(columns={'subject_id': 'source_id'}, inplace=True)
table_b.rename(columns={'subject_id': 'target_id'}, inplace=True)

if len(COLS_A_TO_REMOVE) > 0:
    table_a.drop(columns=COLS_A_TO_REMOVE, inplace=True, errors='ignore')
if len(COLS_B_TO_REMOVE) > 0:
    table_b.drop(columns=COLS_B_TO_REMOVE, inplace=True, errors='ignore')

# Create dictionaries to map idA and idB to their respective indices
idA_to_index = dict(zip(table_a['source_id'], table_a.index))
idB_to_index = dict(zip(table_b['target_id'], table_b.index))

# Replace idA and idB in pairs with their corresponding indices
pairs_train['source_id'] = pairs_train['source_id'].map(idA_to_index)
pairs_train['target_id'] = pairs_train['target_id'].map(idB_to_index)
pairs_val['source_id'] = pairs_val['source_id'].map(idA_to_index)
pairs_val['target_id'] = pairs_val['target_id'].map(idB_to_index)
pairs_test['source_id'] = pairs_test['source_id'].map(idA_to_index)
pairs_test['target_id'] = pairs_test['target_id'].map(idB_to_index)

# pairs are now 'a' + table_a index and 'b' + table_b index
pairs_train['source_id'] = 'a' + pairs_train['source_id'].astype(str)
pairs_train['target_id'] = 'b' + pairs_train['target_id'].astype(str)
pairs_val['source_id'] = 'a' + pairs_val['source_id'].astype(str)
pairs_val['target_id'] = 'b' + pairs_val['target_id'].astype(str)
pairs_test['source_id'] = 'a' + pairs_test['source_id'].astype(str)
pairs_test['target_id'] = 'b' + pairs_test['target_id'].astype(str)


# Display the updated pairs DataFrame
display(pairs_train.head())

display(pairs_train.info())

,source_id,target_id,matching
0,a623,b2861,False
1,a1509,b16350,False
2,a101,b9858,False
3,a838,b13023,False
4,a970,b5511,False


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10905 entries, 0 to 10904
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   source_id  10905 non-null  object
 1   target_id  10905 non-null  object
 2   matching   10905 non-null  bool  
dtypes: bool(1), object(2)
memory usage: 181.2+ KB


None

In [22]:
# Drop idA and idB
table_a = table_a.drop(columns=['source_id'], errors='ignore')
table_b = table_b.drop(columns=['target_id'], errors='ignore')

# Display the first 5 rows of the tableA dataframe
display(table_a.head())

# Display the first 5 rows of the tableB dataframe
display(table_b.head())

,brand,groupname,title,price,shelfdescr,shortdescr,longdescr,imageurl,orig_shelfdescr,orig_shortdescr,orig_longdescr,modelno,shipweight,dimensions
0,Draper,Electronics - General,Draper Infrared Remote Transmitter,58.45,Infrared transmitter. 3-button operation for i...,NaN,DR1143Infrared transmitter. 3-button operation...,http://i.walmartimages.com/i/mp/00/64/10/92/16...,Infrared transmitter. 3-button operation for i...,NaN,DR1143Infrared transmitter. 3-button operation...,121066,2.00,NaN
1,Epson,Monitors,Epson 1500 Hours 200W UHE Projector Lamp ELPLP12,438.84,NaN,Epson ELPLP12 Replacement Lamp,EPSON ELPLP12 1500HRS 200V REPL LAMP FOR LAMP ...,http://i.walmartimages.com/i/p/00/01/03/43/83/...,NaN,Epson ELPLP12 Replacement Lamp,EPSON ELPLP12 1500HRS 200V REPL LAMP FOR LAMP ...,ELPLP12,0.95,6.75 x 5.75 x 5.5 inches
2,Comprehensive,TV Accessories,Comprehensive Two-Piece 75 Precision BNC Jack ...,59.25,Comprehensive s True 75 connectors eliminate i...,NaN,CH1151Comprehensive s True 75 connectors elimi...,http://i.walmartimages.com/i/mp/00/80/84/47/00...,Comprehensive's True 75 connectors eliminate i...,NaN,CH1151Comprehensive's True 75 connectors elimi...,BJ-2C7559,0.05,NaN
3,D-Link,Garden - General,D-Link DCS-1100 Network Camera,99.82,Surveillance Network Camera Built-in Omni-dir...,The D-Link DCS-1100 Network Camera comes with ...,Surveillance Network Camera Built-in Omni-dir...,http://i.walmartimages.com/i/p/00/79/00/69/32/...,<UL><LI>Surveillance/Network Camera</LI><LI>Bu...,The D-Link DCS-1100 Network Camera comes with ...,<UL><LI>Surveillance/Network Camera</LI><LI>Bu...,DCS-1100,NaN,NaN
4,StarTech,Electronics - General,StarTech.com RKPW247015 24 Outlet Power Strip,59.00,Fits most 42U racks and equipment cabinets 10...,The StarTech.com RKPW247015 24 Outlet Power St...,24 Outlet Power Strip solution for your rackmo...,http://i.walmartimages.com/i/p/00/06/50/30/81/...,<ul><li>Fits most 42U racks and equipment cabi...,The StarTech.com RKPW247015 24 Outlet Power St...,<ul><li>24 Outlet Power Strip solution for you...,RKPW247015,5.25,72.5 x 2.5 x 1.5 inches


,brand,modelno,category1,pcategory1,category2,pcategory2,title,listprice,price,prodfeatures,techdetails,shortdescr,longdescr,dimensions,imageurl,itemweight,shipweight,orig_prodfeatures,orig_techdetails,groupname
0,Koss,152132,Headphone Accessories,Accessories Supplies,Equalizers,Home Audio Theater,Koss EQ50 3-Band Stereo Equalizer,19.99,12.65,NaN,10dB boost or cut per channel Unity gain ampli...,The Koss EQ50 is a three band stereo equalizer...,The pocket-size Koss 3-Band Equalizer delivers...,7.6 x 4.8 x 1.4 inches,http://ecx.images-amazon.com/images/I/4124dZb2...,3.2 ounces,1 pounds,NaN,<ul><li>10dB boost or cut per channel</li> <li...,Headphone Accessories Accessories Supplies Equ...
1,Kodak,1163641,Inkjet Printer Ink,Printer Ink Toner,NaN,NaN,Kodak Black Ink Cartridge 10B 1163641,NaN,10.28,NaN,Ships in foil packaging,NaN,Kodak Black Ink Cartridge 10B is a standard bl...,5.3 x 3.2 x 1.2 inches,http://ecx.images-amazon.com/images/I/41%2B2Ea...,1.9 ounces,1 pounds,NaN,<ul><li>Ships in foil packaging</li> </ul>,Inkjet Printer Ink Printer Ink Toner
2,Kingston,KTC-D320/1G,Computers Accessories,Electronics,NaN,NaN,Kingston 128MX64 PC2700 COMPAQ Evo D320 KTC-D3...,60.00,33.75,NaN,DC341A,NaN,1GB - 333MHz DDR333 PC2700 - DDR SDRAM - 184-p...,6.8 x 2.2 x 0.5 inches,http://ecx.images-amazon.com/images/I/21D2P6AN...,1.3 ounces,1 pounds,NaN,<ul><li># DC341A</li> </ul>,Computers Accessories Electronics
3,Kinamax,MS-UES2,Mice,Keyboards Mice Input Devices,NaN,NaN,Kinamax MS-UES2 Mini High Precision USB 3-Butt...,NaN,6.99,NaN,Support Inquiries support sabrent.com Features...,NaN,KINAMAX MS-UES2 Mini High Precision USB 3-Butt...,4.3 x 2.8 x 1.3 inches,http://ecx.images-amazon.com/images/I/31gI-p6d...,1.6 ounces,1 pounds,NaN,<ul><li>Support Inquiries: support@sabrent.com...,Mice Keyboards Mice Input Devices
4,Kensington,K72349US,Mice,Keyboards Mice Input Devices,NaN,NaN,Kensington K72349US Wireless Mouse for Netbooks,24.99,24.00,NaN,Stay plugged in with a nano receiver Web surfi...,NaN,Wireless MOUSE FOR NETBOOKS USBWRLS,4.1 x 3.2 x 1.5 inches,http://ecx.images-amazon.com/images/I/31eIBC9A...,2.1 ounces,1 pounds,NaN,<ul><li>Stay plugged in with a nano receiver</...,Mice Keyboards Mice Input Devices


In [23]:
# Print an exemple of match
print(get_entity(pairs_train.iloc[0]['source_id'], table_a), end='\n\n')
print(get_entity(pairs_train.iloc[0]['target_id'], table_b), end='\n\n')
print(pairs_train.iloc[0]['matching'])

brand                                                          LaCie
groupname                                      Electronics - General
title                LaCie Portable 8x DVD-RW Drive with LightScribe
price                                                           57.6
shelfdescr         LightScribe labeling software USB bus-powered ...
shortdescr         With the LaCie Portable 8x DVD- RW Drive with ...
longdescr          LaCie Portable 8x DVD-RW Drive with LightScrib...
imageurl           http://i.walmartimages.com/i/p/00/09/30/53/79/...
orig_shelfdescr    <li>LightScribe labeling software<li>USB bus-p...
orig_shortdescr    With the LaCie Portable 8x DVD-/+RW Drive with...
orig_longdescr     <b>LaCie Portable 8x DVD-RW Drive with LightSc...
modelno                                                       301910
shipweight                                                      1.04
dimensions                                    7.5 x 7.0 x 2.5 inches
Name: 623, dtype: object

brand   

In [24]:
table_a_serialized = [serialize_entities(entity) for (_, entity) in table_a.iterrows()]
table_b_serialized = [serialize_entities(entity) for (_, entity) in table_b.iterrows()]
table_a_serialized[:5]

["[COL] brand [VAL] Draper [COL] groupname [VAL] Electronics - General [COL] title [VAL] Draper Infrared Remote Transmitter [COL] price [VAL] 58.45 [COL] shelfdescr [VAL] Infrared transmitter. 3-button operation for instant access to up  down and stop functions. Fully compatible with learnable IR master control systems. Receiver sold seperately plugs into the Draper low-voltage control unit LVC-III sold separately . [COL] shortdescr [VAL]  [COL] longdescr [VAL] DR1143Infrared transmitter. 3-button operation for instant access to up  down and stop functions. Fully compatible with learnable IR master control systems. Receiver sold seperately plugs into the Draper low-voltage control unit LVC-III sold separately . [COL] imageurl [VAL] http://i.walmartimages.com/i/mp/00/64/10/92/16/0064109216245_P255045_300X300.jpg [COL] orig_shelfdescr [VAL] Infrared transmitter. 3-button operation for instant access to ''up'', ''down'', and ''stop'' functions. Fully compatible with ''learnable'' IR maste

In [25]:
# Save the serialized tables
with open(os.path.join(DATA_DIR, '1_serialized.pkl'), 'wb') as f:
    pickle.dump(table_a_serialized, f)
with open(os.path.join(DATA_DIR, '2_serialized.pkl'), 'wb') as f:
    pickle.dump(table_b_serialized, f)